In [1]:
import numpy as np
import datetime

x=np.array([[1,1,1],[1,1,-1],[1,-1,1],[1,-1,-1],[-1,1,1],[-1,1,-1],[-1,-1,1],[-1,-1,-1]])
freq=np.array([2,0,7,1,0,3,1,4])

#mean_x=np.mean(x, axis=0)

def energy(w,y):
    o=np.array([y[0],y[1],y[2],y[0]*y[1],y[0]*y[2],y[1]*y[2]])
    return -np.inner(w,o)

#1.Hopfield model
N=x.shape[0]
tot_freq=np.sum(freq)
p=freq/tot_freq


a1,a2,a3,w12,w13,w23=np.zeros(6)
for n in range(N):
    a1+=p[n]*x[n][0]
    a2+=p[n]*x[n][1]
    a3+=p[n]*x[n][0]
    w12+=p[n]*x[n][0]*x[n][1]
    w13+=p[n]*x[n][0]*x[n][2]
    w23+=p[n]*x[n][1]*x[n][2]
    
w=[a1,a2,a3,w12,w13,w23]

q=np.zeros(N)
for n in range(N):
    q[n]=np.exp(-energy(w,x[n]))
q=q/np.sum(q)    
    
print('Hopfield model')    
print('true pred')
for n in range(N):
    print('%.2f %.2f' % (p[n], q[n]))

print(datetime.datetime.now())    
#2. Boltzmann machine
alpha = 0.1 #Learning rate

a1,a2,a3,w12,w13,w23=np.random.rand(6)
#print('%.2f %.2f %.2f %.2f %.2f %.2f' % (a1,a2,a3,w12,w13,w23))
for k in range(100):
    w=[a1,a2,a3,w12,w13,w23]
    for n in range(N):
        q[n]=np.exp(-energy(w,x[n]))
    q=q/np.sum(q)
    o_p=np.zeros(6)
    o_q=np.zeros(6)
    for n in range(N):
        o=np.array([x[n][0],x[n][1],x[n][2],x[n][0]*x[n][1],x[n][0]*x[n][2],x[n][1]*x[n][2]])
        o_p+=p[n]*o
        o_q+=q[n]*o
    dKL=o_p-o_q

    a1,a2,a3,w12,w13,w23=w+alpha*dKL
    #print('%.2f %.2f %.2f %.2f %.2f %.2f' % (a1,a2,a3,w12,w13,w23))
    
print('Boltzmann machine')  
print('true pred')
for n in range(N):
    print('%.2f %.2f' % (p[n], q[n]))    
    
    
print(datetime.datetime.now())
#3. Restricted Boltzmann machine (RBM)
xx=x
hh=np.array([[1,1],[1,-1],[-1,1],[-1,-1]])
M=hh.shape[0]

def rbm_energy(a, b, w, v, h):
    e=-(np.inner(a,v)+np.inner(b,h))
    np.reshape(v,(v.shape[0],1))
    np.reshape(h,(h.shape[0],1))
    e-=np.matmul(v.T,np.matmul(w,h))
    return e

#def rbm_energy(a, b, w, v, h):
#    e=0.;
#    for i in range(v.shape[0]):
#        e-=a[i]*v[i]
#    for j in range(h.shape[0]):
#        e-=b[j]*h[j]
#    for i in range(v.shape[0]):
#        for j in range(h.shape[0]):
#            e-=w[i][j]*v[i]*h[j]
#    return e

v_dim=xx.shape[1]
h_dim=hh.shape[1]
a=np.random.rand(v_dim)
b=np.random.rand(h_dim)
ww=np.random.rand(v_dim,h_dim)

pp=np.zeros((N,M)) #pp(v,h)=p(v)q(h|v)
qq=np.zeros((N,M)) #qq(v,h)=q(v,h)=q(v)q(h|v)

for k in range(100):

    z_q=np.zeros(N)
    for n in range(N):
        for m in range(M):
            z_q[n]+=np.exp(-rbm_energy(a,b,ww,xx[n],hh[m]))
    z=np.sum(z_q)        
        
    for n in range(N):
        for m in range(M):
            pp[n][m]=p[n]*np.exp(-rbm_energy(a,b,ww,xx[n],hh[m]))/z_q[n]
            qq[n][m]=np.exp(-rbm_energy(a,b,ww,xx[n],hh[m]))/z
            
    da=np.zeros(v_dim)  
    for i in range(v_dim):
        for n in range(N):
            for m in range(M):
                da[i]+=xx[n][i]*pp[n][m]-xx[n][i]*qq[n][m]
        a[i]+=alpha*da[i]
                
    db=np.zeros(h_dim)   
    for j in range(h_dim):
        for n in range(N):
            for m in range(M):
                db[j]+=hh[m][j]*pp[n][m]-hh[m][j]*qq[n][m]
        b[j]+=alpha*db[j]       
                
    dww=np.zeros((v_dim,h_dim))       
    for i in range(v_dim):
        for j in range(h_dim):
            for n in range(N):
                for m in range(M):
                    dww[i][j]+=xx[n][i]*hh[m][j]*pp[n][m]-xx[n][i]*hh[m][j]*qq[n][m]
            ww[i][j]+=alpha*dww[i][j]
            
q=np.zeros(N)
for n in range(N):
    for m in range(M):
        q[n]+=qq[n][m] #marginalizing h
                                 
print('Restricted Boltzmann machine')  
print('true pred')
for n in range(N):
    print('%.2f %.2f' % (p[n], q[n]))                                      
  
print(datetime.datetime.now())
#4. Restricted Boltzmann machine by using CD1 (RBM)
xx=x
hh=np.array([[1,1],[1,-1],[-1,1],[-1,-1]])
M=hh.shape[0]
v0=x
h0=np.zeros((N, h_dim))
v1=np.zeros((N, v_dim))
h1=np.zeros((N, h_dim))

alpha=0.1

def rbm_energy(a, b, w, v, h):
    e=-(np.inner(a,v)+np.inner(b,h))
    np.reshape(v,(v.shape[0],1))
    np.reshape(h,(h.shape[0],1))
    e-=np.matmul(v.T,np.matmul(w,h))
    return e

#def rbm_energy(a, b, w, v, h):
#    e=0.;
#    for i in range(v.shape[0]):
#        e-=a[i]*v[i]
#    for j in range(h.shape[0]):
#        e-=b[j]*h[j]
#    for i in range(v.shape[0]):
#        for j in range(h.shape[0]):
#            e-=w[i][j]*v[i]*h[j]
#    return e

#v_dim=xx.shape[1]
#h_dim=hh.shape[1]
a=np.random.rand(v_dim)
b=np.random.rand(h_dim)
ww=np.random.rand(v_dim,h_dim)

for k in range(100):

    #v0 --> h0 by using p(h0|v0)
    for n in range(N):
        temph=np.ones(h_dim)
        for j in range(h_dim):
            prob1=np.exp(-rbm_energy(a,b,ww,v0[n],temph))
            temph1=temph
            temph1[j]=-temph1[j]
            prob2=np.exp(-rbm_energy(a,b,ww,v0[n],temph1))
            prob=prob1/(prob1+prob2)
            h0[n][j]=1
            if(prob < np.random.rand()):
                h0[n][j]=-1
                
    #h0 --> v1 by using p(v1|h0)            
    for n in range(N):
        tempv=np.ones(v_dim)
        for i in range(v_dim):
            prob1=np.exp(-rbm_energy(a,b,ww,tempv,h0[n]))
            tempv1=tempv
            tempv1[i]=-tempv1[i]
            prob2=np.exp(-rbm_energy(a,b,ww,tempv1,h0[n]))
            prob=prob1/(prob1+prob2)
            v1[n][i]=1
            if(prob < np.random.rand()):
                v1[n][i]=-1

    #v1 --> h1 by using p(h1|v1)            
    for n in range(N):
        temph=np.ones(h_dim)
        for j in range(h_dim):
            prob1=np.exp(-rbm_energy(a,b,ww,v1[n],temph))
            temph1=temph
            temph1[j]=-temph1[j]
            prob2=np.exp(-rbm_energy(a,b,ww,v1[n],temph1))
            prob=prob1/(prob1+prob2)
            h1[n][j]=1
            if(prob < np.random.rand()):
                h1[n][j]=-1                    
                           
                     
    da=np.zeros(v_dim)
    db=np.zeros(h_dim)
    dww=np.zeros((v_dim,h_dim))
    
    for i in range(v_dim):
        for n in range(N):
            da[i]+=(v0[n][i]-v1[n][i])*p[n] #p[n] sample weight
            
    for j in range(h_dim):
        for n in range(N):
            db[j]+=(h0[n][j]-h1[n][j])*p[n]        
                
    for i in range(v_dim):
        for j in range(h_dim):
            for n in range(N):
                dww[i][j]+=(v0[n][i]*h0[n][j]-v1[n][i]*h1[n][j])*p[n]
    
    for i in range(v_dim):
        a[i]+=alpha*da[i]
    for j in range(h_dim):
        b[j]+=alpha*db[j]
             
    for i in range(v_dim):
        for j in range(h_dim):
            ww[i][j]+=alpha*dww[i][j]
         
    #print(a,b,ww)    
qq=np.zeros((N,M)) #qq(v,h)=q(v,h)=q(v)q(h|v)
  
z_q=np.zeros(N)
for n in range(N):
    for m in range(M):
        z_q[n]+=np.exp(-rbm_energy(a,b,ww,xx[n],hh[m]))
z=np.sum(z_q)        
        
for n in range(N):
    for m in range(M):
        qq[n][m]=np.exp(-rbm_energy(a,b,ww,xx[n],hh[m]))/z

q=np.zeros(N)
for n in range(N):
    for m in range(M):
        q[n]+=qq[n][m]
        
print('RBM with CD')  
print('true pred')
for n in range(N):
    print('%.2f %.2f' % (p[n], q[n])) 
    
print(datetime.datetime.now())

Hopfield model
true pred
0.11 0.08
0.00 0.02
0.39 0.50
0.06 0.05
0.00 0.02
0.17 0.13
0.06 0.05
0.22 0.13
2019-06-26 09:13:50.161290
Boltzmann machine
true pred
0.11 0.10
0.00 0.02
0.39 0.40
0.06 0.04
0.00 0.02
0.17 0.15
0.06 0.04
0.22 0.24
2019-06-26 09:13:50.205435
Restricted Boltzmann machine
true pred
0.11 0.09
0.00 0.02
0.39 0.40
0.06 0.04
0.00 0.02
0.17 0.15
0.06 0.04
0.22 0.23
2019-06-26 09:13:50.739627
RBM with CD
true pred
0.11 0.12
0.00 0.02
0.39 0.39
0.06 0.05
0.00 0.01
0.17 0.16
0.06 0.03
0.22 0.22
2019-06-26 09:13:50.944972
